<a href="https://colab.research.google.com/github/NiloofarSoltani-2/Fater_Consumption_Analysis/blob/main/Github_Fater_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
from google.colab import files
import pandas as pd
import re
import os
from google.colab import files # for exporting dataset as an excel file
import seaborn as sns
import matplotlib.pyplot as plt
!pip install pandas openpyxl

## **Merging 66 Excel Files**

*   **List All The Columns In Both DATASETS**
*   **Change Columns name (specially Date columns ((key columns)))**
*   **Export Dataset Seperatly**

In [3]:
# Define the directory containing your Excel files
directory_path = '/content/sample_data'  # Replace with your directory path

# List all Excel files in the directory
excel_files = [f for f in os.listdir(directory_path) if f.endswith('.xls')]

# Create an empty list to store individual DataFrames
df_list = []

# Loop over each file and process it
for file in excel_files:
    # Build the full file path
    file_path = os.path.join(directory_path, file)

    # Read the Excel file
    df = pd.read_excel(file_path)

    # Add a new column for the source file
    df['source_file'] = file

    # Rename 'Tempo' column to 'tempo_standardized'
    df.rename(columns={'Tempo': 'Date'}, inplace=True)


    # Dynamically find and rename the kWh column using regex
    kwh_col = next((col for col in df.columns if re.search(r'kwh|[kwh]|[Kwh]|EATT', col, re.IGNORECASE)), None)
    if kwh_col:
        df.rename(columns={kwh_col: 'kwh_standardized'}, inplace=True)
    else:
        print(f"Warning: Could not find kWh column in file: {file}")
        # Handle the case where no kWh column is found (e.g., skip the file, assign a default value)

    # Append the DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(df_list, ignore_index=True)

# Now select the standardized column names
combined_df = combined_df[['Date', 'kwh_standardized', 'source_file']]



# Rename columns to the desired format (if needed)
#combined_df = combined_df[['tempo', 'kwh', 'source_file']]

# # Set the option to display all rows (if you need to see all records)
# pd.set_option('display.max_rows', None)

# Display the combined dataset
print(combined_df)

# Optionally, save the combined dataset to a new Excel file
combined_df.to_excel('/content/sample_data/outputs/combined_dataset_66excels.xlsx', index=False)

# Check the structure of the dataset
print(combined_df.info())  # Columns, data types, non-null counts

                       Date  kwh_standardized      source_file
0       2018-09-05 00:00:00      1.087410e+03        FIX_4.xls
1       2018-09-06 00:00:00      1.064330e+03        FIX_4.xls
2       2018-09-07 00:00:00      1.692580e+03        FIX_4.xls
3       2018-09-08 00:00:00      6.876600e+02        FIX_4.xls
4       2018-09-09 00:00:00      0.000000e+00        FIX_4.xls
...                     ...               ...              ...
147192  2024-10-26 00:00:00      5.154900e+02  CTRL_IDRICA.xls
147193  2024-10-27 00:00:00      4.972800e+02  CTRL_IDRICA.xls
147194  2024-10-28 00:00:00      5.415500e+02  CTRL_IDRICA.xls
147195                Media      6.020933e+02  CTRL_IDRICA.xls
147196               Totale      1.392040e+06  CTRL_IDRICA.xls

[147197 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147197 entries, 0 to 147196
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date  

# **Each Files Has How Many Rows?** **plot the output**


*   ***output is a dic***



In [4]:
# @title
# Group by 'Utility' and count the rows for each utility
utility_row_counts = combined_df.groupby('source_file').size().reset_index(name='Row Count')
print(utility_row_counts)


# Count how many have records lower than 2314
lower_than_2314 = utility_row_counts[utility_row_counts['Row Count'] < 2314]
count_lower = len(lower_than_2314)
print(f"Number of source_file columns with records lower than 2314: {count_lower}")


# Count how many have exactly 2314 records
equal_to_2314 = utility_row_counts[utility_row_counts['Row Count'] == 2314]
count_equal = len(equal_to_2314)
print(f"Number of source_file columns with exactly 2314 records: {count_equal}")

# Get a list of source_file columns with records lower than 2314
columns_with_lower = lower_than_2314['source_file'].tolist()
print("List of source_file columns with records lower than 2314:")
print(columns_with_lower)


# Create a dictionary with column names and their record counts
source_file_dict = utility_row_counts.set_index('source_file')['Row Count'].to_dict()
print("Dictionary of source_file columns and their record counts:")
print(source_file_dict)



                   source_file  Row Count
0                 CAMFIL_2.xls       2314
1                 CAMFIL_5.xls       1444
2             CENT_TERMICA.xls       2314
3                CHILLER_1.xls       2314
4                CHILLER_2.xls       2314
..                         ...        ...
61         UTENZE_TECNOLOG.xls       2303
62     UTENZE_TECN_LOTTO_B.xls       2218
63  UTILITIES_GLOBAL_12345.xls       2314
64                   WIPES.xls       2314
65              chiller_1c.xls       2303

[66 rows x 2 columns]
Number of source_file columns with records lower than 2314: 30
Number of source_file columns with exactly 2314 records: 36
List of source_file columns with records lower than 2314:
['CAMFIL_5.xls', 'CHILLER_3_LOTTO_C.xls', 'CHILLER_4_LOTTO_C.xls', 'COMPRESSORI.xls', 'COMPR_VUOTO.xls', 'COV.xls', 'FAX_15.xls', 'FIX_2.xls', 'FIX_4.xls', 'FNL_3.xls', 'FNL_8.xls', 'GEN_LOTTO_B.xls', 'HYBRID.xls', 'IDROFILTRO_2C.xls', 'IDROFILTRO_4.xls', 'MAG_PROD_FINITO_1.xls', 'MAG_PROD_F

# **BEST WAY OF VISUALIZATION**

In [13]:
# @title
import plotly.express as px

# Create interactive bar chart
fig = px.bar(utility_row_counts, x='source_file', y='Row Count', title='Row Count for Each Utility')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()


# **Upload 8 Excels Files**

In [6]:
# Upload the file
uploaded = files.upload()

Saving msu_pe_1617.XLSX to msu_pe_1617.XLSX
Saving msu_pe_1718.XLSX to msu_pe_1718.XLSX
Saving msu_pe_1819.XLSX to msu_pe_1819.XLSX
Saving msu_pe_1920.XLSX to msu_pe_1920.XLSX
Saving msu_pe_2021.XLSX to msu_pe_2021.XLSX
Saving msu_pe_2122.XLSX to msu_pe_2122.XLSX
Saving msu_pe_2223.XLSX to msu_pe_2223.XLSX
Saving msu_pe_2324.XLSX to msu_pe_2324.XLSX


# **Merging 8 Excels Files**


*   **List All The Columns In Both DATASETS**
*   **Change Columns name (specially Date columns ((key columns)))**
*   **Export Dataset Seperatly**



In [11]:
# Initialize an empty list to store DataFrames
dataframes = []

# Loop through the uploaded files
for file_name in uploaded.keys():
    # Read each Excel file and append to the list
    df = pd.read_excel(file_name)
    dataframes.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Optionally, save the combined dataset to a new Excel file
merged_df.to_excel('/content/sample_data/combined_dataset_8excels.xlsx', index=False)

# Check the structure of the dataset
print(merged_df.info())  # Columns, data types, non-null counts

# sum of excel records are 71123

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71395 entries, 0 to 71394
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Sem.                 0 non-null      float64       
 1   Legenda              71395 non-null  object        
 2   Linea di produzione  71363 non-null  object        
 3   Materiale            71200 non-null  float64       
 4   Versione di prod.    0 non-null      float64       
 5   Data prod.           71200 non-null  datetime64[ns]
 6   Turno                0 non-null      float64       
 7   %TSR                 71395 non-null  float64       
 8   % Scarto             71395 non-null  float64       
 9   % Sched. Util.       71395 non-null  float64       
 10  % Rate Util.         71395 non-null  float64       
 11  % CU                 71395 non-null  float64       
 12  Resa                 71395 non-null  float64       
 13  Pz / min             71395 non-

In [12]:
# Rename specific columns
merged_df = merged_df.rename(columns={
    'Data prod.': 'Date'
})